# Promethee V
This module shows an example of how to use the "Promethee V" module

## Definition of inputs and problem formalization

In [19]:
import pandas as pd
from core.constraint import Constraint, Relation
from modular_parts.preference import compute_preference_indices
from modular_parts.flows import calculate_promethee_outranking_flows, calculate_net_outranking_flows
from core.enums import Direction, GeneralCriterion, FlowType
from modular_parts.choice import compute_decision


alternatives = [f"a{i}" for i in range(1, 8)]
profiles = [f"p{i}" for i in range(1, 4)]
criteria = [f"c{i}" for i in range(1, 4)]
criteria_directions = pd.Series([Direction.MAX, Direction.MIN, Direction.MAX], index=criteria)
criteria_weights = pd.Series([0.3, 0.2, 0.5], index=criteria)
generalised_criteria = pd.Series([GeneralCriterion.U_SHAPE,
                                  GeneralCriterion.V_SHAPE_INDIFFERENCE,
                                  GeneralCriterion.USUAL], index=criteria)
preference_thresholds = pd.Series([2, 10, None], index=criteria)
indifference_thresholds = pd.Series([1, 5, None], index=criteria)
standard_deviations = pd.Series([None, None, None], index=criteria) # None, because we do not use GeneralCriterion.GAUSSIAN

alternatives_performances = pd.DataFrame([[15, 83, 21],
                                          [10, 90, 15],
                                          [11, 75, 20],
                                          [18, 59, 20],
                                          [17, 60, 28],
                                          [22, 44, 15],
                                          [13, 62, 22]], index=alternatives, columns=criteria)

constraints1 = [Constraint([0.6, 0.9, 1.5, 1.2, 0.3, 1.2, 0.4], Relation.LEQ, 3)]
constraints2 = [Constraint([0.6, 0.9, 1.5, 1.2, 0.3, 1.2, 0.4], Relation.GEQ, 3)]
constraints3 = [Constraint([0.4, 0.9, 1.5, 1.2, 0.3, 1.2, 0.4], Relation.EQ, 3)]

## Required data

### Required preferences

In [20]:
aggregated_alternatives_preferences, _ = compute_preference_indices(alternatives_performances,
                                                                    preference_thresholds,
                                                                    indifference_thresholds,
                                                                    standard_deviations,
                                                                    generalised_criteria,
                                                                    criteria_directions,
                                                                    criteria_weights)
aggregated_alternatives_preferences

,a1,a2,a3,a4,a5,a6,a7
a1,0.00,0.88,0.8,0.5,0.0,0.5,0.3
a2,0.00,0.00,0.0,0.0,0.0,0.0,0.0
a3,0.12,0.70,0.0,0.0,0.0,0.5,0.0
a4,0.50,1.00,0.5,0.0,0.0,0.5,0.3
a5,1.00,1.00,1.0,0.5,0.0,0.5,0.8
a6,0.50,0.50,0.5,0.5,0.5,0.0,0.5
a7,0.70,1.00,1.0,0.5,0.0,0.5,0.0


### Required flows

In [21]:
alternatives_flows = calculate_promethee_outranking_flows(aggregated_alternatives_preferences,
                                                          flow_type=FlowType.BASIC)
alternatives_net_flows = calculate_net_outranking_flows(alternatives_flows)
alternatives_net_flows

a1    0.026667
a2   -0.846667
a3   -0.413333
a4    0.133333
a5    0.716667
a6    0.083333
a7    0.300000
Name: Net outranking flow, dtype: float64

## Promethee V

In [22]:
compute_decision(alternatives_net_flows, constraints1)

0    a1
1    a4
2    a5
3    a7
Name: chosen alternatives, dtype: object

In [23]:
compute_decision(alternatives_net_flows, constraints2)

0    a1
1    a4
2    a5
3    a6
4    a7
Name: chosen alternatives, dtype: object

In [24]:
compute_decision(alternatives_net_flows, constraints3)

0    a3
1    a4
2    a5
Name: chosen alternatives, dtype: object